# Dialogue Analysis Agent 테스트 (Production Level)

대화 분석 에이전트 (Level 1) 테스트 노트북

## 역할: "Script Supervisor" (스크립트 감독)
- 대화 패턴 분석
- 화자별 말투 특성 추출
- 대화 관계 (Neo4j 엣지용)
- 숨겨진 의미 (Subtext) 분석

> **Note**: 에이전트가 직접 `neo4j_edges` 필드를 반환합니다. (Machine-Readable JSON)

In [1]:
import sys, os, json, asyncio
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path: sys.path.insert(0, project_root)
from dotenv import load_dotenv
load_dotenv(os.path.join(project_root, '.env'))
print(f"Project root: {project_root}")

Project root: c:\jungle\weapon\sto-link-AI-backend


In [2]:
SAMPLE_STORY = """
"우리가 이곳에서 그를 만날 수 있을까?" 하나가 조심스럽게 물었다.

"오랜만이군, 서진." 이민호가 차갑게 말했다. "아직도 그 낡은 검을 들고 다니는군."

서진은 눈을 똑바로 바라보며 대답했다. "왜 마을을 배신했어, 민호? 우리가 함께 자란 마을이잖아."

이민호는 씁쓸하게 웃었다. "넌 모르지. 마을 장로들이 내 가족에게 무슨 짓을 했는지. 
그들은 내 아버지를..."

하나가 두 사람 사이로 걸어나왔다. "싸우기 전에 대화를 해봐요. 진실을 알아야 해요."

"진실?" 이민호가 비웃었다. "진실은 항상 강자의 편이지."
"""

EXTRACTED_CHARACTERS = [
    {"name": "서진", "role": "주인공"},
    {"name": "이민호", "role": "적대자"},
    {"name": "하나", "role": "조력자"},
    {"name": "박서연", "role": "조력자"}
]

def create_base_state(story=SAMPLE_STORY):
    return {"content": story, "project_id": "test-001", "document_id": "doc-001", 
            "job_id": "job-001", "callback_url": "http://localhost:8080/internal/callback",
            "extracted_characters": EXTRACTED_CHARACTERS,
            "analyzed_dialogues": {}, "messages": [], "errors": []}

def run_async(coro):
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            import nest_asyncio; nest_asyncio.apply()
            return loop.run_until_complete(coro)
        return asyncio.run(coro)
    except: return asyncio.run(coro)

## 1. Dialogue Analysis 실행

In [3]:
from app.agents.extraction.dialogue import dialogue_analysis_node

async def test_dialogue_analysis():
    print("💬 Dialogue Analysis (Production Level) 테스트...")
    state = create_base_state()
    return await dialogue_analysis_node(state)

result = run_async(test_dialogue_analysis())
dialogues = result.get("analyzed_dialogues", {})

print(f"\n✅ 분석 완료:")
print(f"   - key_dialogues: {len(dialogues.get('key_dialogues', []))}개")
print(f"   - speech_patterns: {len(dialogues.get('speech_patterns', []))}개")
print(f"   - dialogue_relationships: {len(dialogues.get('dialogue_relationships', []))}개")
print(f"   - neo4j_edges: {len(dialogues.get('neo4j_edges', []))}개 (Machine-Readable)")

💬 Dialogue Analysis (Production Level) 테스트...
[DIALOGUE] Available characters: ['서진', '이민호', '하나', '박서연']

✅ 분석 완료:
   - key_dialogues: 6개
   - speech_patterns: 3개
   - dialogue_relationships: 6개
   - neo4j_edges: 6개 (Machine-Readable)


## 2. Neo4j-Ready JSON 출력 (Machine-Readable)

에이전트가 직접 반환하는 `neo4j_edges` 필드입니다. `json.loads()` 한 번으로 바로 사용 가능합니다.

In [4]:
# 에이전트에서 직접 반환한 neo4j_edges (별도 변환 불필요!)
neo4j_edges = dialogues.get("neo4j_edges", [])

print("="*70)
print("🔗 Neo4j-Ready JSON (Machine-Readable) - Agent 직접 반환")
print("="*70)
print(json.dumps(neo4j_edges, ensure_ascii=False, indent=2))
print(f"\n📊 Total edges: {len(neo4j_edges)}")

🔗 Neo4j-Ready JSON (Machine-Readable) - Agent 직접 반환
[
  {
    "source": "하나",
    "target": "서진",
    "relationship_type": "SPEAKS_TO",
    "attributes": {
      "formality": "formal",
      "power": "subordinate",
      "intimacy": 7
    }
  },
  {
    "source": "이민호",
    "target": "서진",
    "relationship_type": "SPEAKS_TO",
    "attributes": {
      "formality": "informal",
      "power": "equal",
      "intimacy": 2
    }
  },
  {
    "source": "서진",
    "target": "이민호",
    "relationship_type": "SPEAKS_TO",
    "attributes": {
      "formality": "informal",
      "power": "equal",
      "intimacy": 3
    }
  },
  {
    "source": "이민호",
    "target": "서진",
    "relationship_type": "SPEAKS_TO",
    "attributes": {
      "formality": "informal",
      "power": "equal",
      "intimacy": 2
    }
  },
  {
    "source": "하나",
    "target": "서진",
    "relationship_type": "SPEAKS_TO",
    "attributes": {
      "formality": "formal",
      "power": "subordinate",
      "intimacy": 7
    }


## 3. Human-Readable 출력 (디버깅용)

In [5]:
print("="*70)
print("👤 Human-Readable 출력 (디버깅용)")
print("="*70)

for edge in neo4j_edges:
    source = edge.get('source', 'N/A')
    target = edge.get('target', 'N/A')
    attrs = edge.get('attributes', {})
    
    print(f"\n{source} → {target}")
    print(f"   격식: {attrs.get('formality')} | 권력: {attrs.get('power')} | 친밀도: {attrs.get('intimacy')}/10")

👤 Human-Readable 출력 (디버깅용)

하나 → 서진
   격식: formal | 권력: subordinate | 친밀도: 7/10

이민호 → 서진
   격식: informal | 권력: equal | 친밀도: 2/10

서진 → 이민호
   격식: informal | 권력: equal | 친밀도: 3/10

이민호 → 서진
   격식: informal | 권력: equal | 친밀도: 2/10

하나 → 서진
   격식: formal | 권력: subordinate | 친밀도: 7/10

하나 → 이민호
   격식: formal | 권력: subordinate | 친밀도: 5/10


## 4. 참조 무결성 검사 (Referential Integrity)

In [6]:
valid_names = {c["name"] for c in EXTRACTED_CHARACTERS}

print("="*70)
print("🔍 Referential Integrity Check")
print("="*70)
print(f"Valid names from Character Agent: {valid_names}")

integrity_errors = []
for edge in neo4j_edges:
    if edge.get('source') not in valid_names:
        integrity_errors.append(f"Source '{edge.get('source')}' not in Character list")
    if edge.get('target') not in valid_names:
        integrity_errors.append(f"Target '{edge.get('target')}' not in Character list")

if integrity_errors:
    print(f"\n❌ INTEGRITY ERRORS ({len(integrity_errors)}):")
    for err in integrity_errors:
        print(f"   - {err}")
else:
    print("\n✅ All names match Character Agent output - Referential Integrity OK!")

🔍 Referential Integrity Check
Valid names from Character Agent: {'박서연', '이민호', '서진', '하나'}

✅ All names match Character Agent output - Referential Integrity OK!


## 5. Spring Boot Callback 데이터 구조

In [7]:
# Spring Boot Callback 전체 데이터 구조
callback_data = {
    "dialogue_analysis": {
        "key_dialogues": dialogues.get("key_dialogues", []),
        "speech_patterns": dialogues.get("speech_patterns", []),
        "neo4j_edges": neo4j_edges  # 바로 사용 가능!
    }
}

print("="*70)
print("📤 Spring Boot Callback 데이터 구조")
print("="*70)

# 전체 JSON 출력
full_json = json.dumps(callback_data, ensure_ascii=False, indent=2)
print(full_json[:3000])
if len(full_json) > 3000:
    print("... (truncated)")

📤 Spring Boot Callback 데이터 구조
{
  "dialogue_analysis": {
    "key_dialogues": [
      {
        "dialogue_id": "D001",
        "participants": [
          "하나",
          "서진"
        ],
        "content": "우리가 이곳에서 그를 만날 수 있을까?",
        "significance": "불안함과 걱정을 드러내는 첫 대사",
        "subtext": "이민호에 대한 두려움"
      },
      {
        "dialogue_id": "D002",
        "participants": [
          "이민호",
          "서진"
        ],
        "content": "오랜만이군, 서진. 아직도 그 낡은 검을 들고 다니는군.",
        "significance": "과거 관계와 현재 적대감을 암시",
        "subtext": "서진을 깎아내리는 의도"
      },
      {
        "dialogue_id": "D003",
        "participants": [
          "서진",
          "이민호"
        ],
        "content": "왜 마을을 배신했어, 민호? 우리가 함께 자란 마을이잖아.",
        "significance": "서진의 실망과 의문 표현",
        "subtext": "마을에 대한 애착과 배신감"
      },
      {
        "dialogue_id": "D004",
        "participants": [
          "이민호",
          "서진"
        ],
        "content": "넌 모르지. 마을 장로들이 내 가족에게 무슨 짓을 했는지. 그들은 내 아버지를...",
     